In [3]:
import os

In [1]:
%pwd

'e:\\PWSkills\\MLFlow-End-to-end-project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\PWSkills\\MLFlow-End-to-end-project'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Vaishali-Sharma01/MLFlow-End-to-end-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Vaishali-Sharma01"
os.environ["MLFLOW_TRACKING_PASSWORD"]="56073c3f3bcd51f47998d6fff39f076cc0ea7815"

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict  #params.yaml
    target_column: str
    mlflow_uri: str
    

In [11]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config =read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Vaishali-Sharma01/MLFlow-End-to-end-project.mlflow"
        )

        return model_evaluation_config

In [17]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config=config

    def evaluate_metrics(self,actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)

        return rmse,mae,r2

    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_data=model.predict(test_x)
            (rmse, mae, r2) = self.evaluate_metrics(test_y, predicted_data)

            #storing metrics
            model_scores={"rmse":rmse,"mae":mae,"r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=model_scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            if tracking_url_type_store!="file":

                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNetLinearRegressorModel")

            else:
                mlflow.sklearn.log_model(model,"model")
            




In [18]:
try:
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evaluator=ModelEvaluation(config=model_evaluation_config)
    model_evaluator.log_into_mlflow()

except Exception as e:
    raise e

[2023-11-08 01:14:59,165: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-08 01:14:59,170: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-08 01:14:59,178: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-08 01:14:59,186: INFO: common: created directory at: artifacts]
[2023-11-08 01:14:59,186: INFO: common: created directory at: artifacts/model_evaluation]
[2023-11-08 01:15:02,892: INFO: common: json file saved at: artifacts\model_evaluation\metric.json]


Successfully registered model 'ElasticNetLinearRegressorModel'.
2023/11/08 01:15:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetLinearRegressorModel, version 1
Created version '1' of model 'ElasticNetLinearRegressorModel'.
